In [127]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 3 para pegar movimentações de processos na busca do 1º Grau 
# Com o arquivo gerado no programa 2 entra em cada página de processo
# e capta informações finais do processo, incluindo as movimentações 
#

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

In [128]:
df_processos = pd.read_excel('resultados/processos_teste_tjsp_23_abril_segunda_fase.xlsx', Sheet="Sheet1")

In [129]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 12 columns):
num_processo         1097 non-null object
link                 1097 non-null object
tipo                 1097 non-null object
parte_especie        1097 non-null object
nome_parte           1097 non-null object
data                 1097 non-null datetime64[ns]
foro                 1097 non-null object
assunto              1097 non-null object
mais_detalhes        358 non-null object
data_distribuicao    1097 non-null object
magistrado           1087 non-null object
valor_acao           1065 non-null object
dtypes: datetime64[ns](1), object(11)
memory usage: 102.9+ KB


In [130]:
contagem = 0

In [131]:
for num, row in df_processos.iterrows():
    time.sleep(1)
    
    link = row['link']
    num_processo = row['num_processo']
    nome_parte = row['nome_parte']
    tipo = row['tipo']
    parte_especie = row['parte_especie']
    foro = row['foro']
    assunto = row['assunto']
    mais_detalhes = row['mais_detalhes']
    magistrado = row['magistrado']
    valor_acao = row['valor_acao']
    
    print(link)
    driver = webdriver.Chrome()
    driver.get(link)
    
    try:      
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#linkmovimentacoes'))).click()
        time.sleep(1)
        pd.options.mode.chained_assignment = None
        tables = pd.read_html(driver.find_element_by_css_selector('#divLinksTituloBlocoMovimentacoes + table').get_attribute('outerHTML'))
        result = tables[0]
        result.drop(['Unnamed: 1'], axis=1, inplace = True)
        driver.quit()
        result['num_processo'] = num_processo
        result['link'] = link
        result['nome_parte'] = nome_parte
        result['tipo'] = tipo
        result['parte_especie'] = parte_especie
        result['foro'] = foro
        result['assunto'] = assunto
        result['mais_detalhes'] = mais_detalhes
        result['magistrado'] = magistrado
        result['valor_acao'] = valor_acao
        
        
    except TimeoutException or NoSuchElementException:
        r = requests.get(link, verify = False)
        tables = pd.read_html(r.content)
        result = tables[4].head(6)
        driver.quit()
        result['num_processo'] = num_processo
        result['link'] = link
        result['nome_parte'] = nome_parte
        result['tipo'] = tipo
        result['parte_especie'] = parte_especie
        result['foro'] = foro
        result['assunto'] = assunto
        result['mais_detalhes'] = mais_detalhes
        result['magistrado'] = magistrado
        result['valor_acao'] = valor_acao
        
   
    
    if contagem == 0:
        df_movimenta = result
    else:
        df_movimenta = pd.concat([df_movimenta, result], sort=True)
        
    contagem = contagem + 1

https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=1H000CAWJ0000&processo.foro=53&paginaConsulta=1&conversationId=&dadosConsulta.localPesquisa.cdLocal=53&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta=Folha%20da%20Manha&uuidCaptcha=&pbEnviar=Pesquisar
https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=2S00111460000&processo.foro=100&paginaConsulta=1&conversationId=&dadosConsulta.localPesquisa.cdLocal=100&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta=Folha%20da%20Manha&uuidCaptcha=&pbEnviar=Pesquisar
https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=2S00110100000&processo.foro=100&paginaConsulta=1&conversationId=&dadosConsulta.localPesquisa.cdLocal=100&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta=Folha%20da%20Manha&uuidCaptcha=&pbEnviar=Pesquisar
https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=2S0010HU80000&processo.foro=100&paginaConsulta=1&convers

/Users/abraji/Code/abraji/ctrl_x/lib/python3.7/site-packages/pandas/core/indexes/api.py:117: RuntimeWarning: '<' not supported between instances of 'tuple' and 'int', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


TypeError: Expected tuple, got str

In [132]:
df_movimenta.tail()

,Data,Movimento,Unnamed: 1,assunto,foro,link,magistrado,mais_detalhes,nome_parte,num_processo,parte_especie,tipo,valor_acao,0
11,06/03/2018,Mandado Expedido Mandado nº: 021.2018/014507-...,NaN,Indenizaçao por Dano Moral,Setor Unificado de Cartas Precatórias Cíveis,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Edna Kyoko Kano,NaN,FACEBOOK SERVIÇOS ONLINE DO BRASIL LTDA,0009178-80.2018.8.26.0021,Reqdo,Indenizaçao por Dano Moral,NaN,NaN
12,06/03/2018,Recebidos os Autos do Distribuidor local,NaN,Indenizaçao por Dano Moral,Setor Unificado de Cartas Precatórias Cíveis,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Edna Kyoko Kano,NaN,FACEBOOK SERVIÇOS ONLINE DO BRASIL LTDA,0009178-80.2018.8.26.0021,Reqdo,Indenizaçao por Dano Moral,NaN,NaN
13,05/03/2018,Remetidos os Autos ao Cartório (movimentação e...,NaN,Indenizaçao por Dano Moral,Setor Unificado de Cartas Precatórias Cíveis,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Edna Kyoko Kano,NaN,FACEBOOK SERVIÇOS ONLINE DO BRASIL LTDA,0009178-80.2018.8.26.0021,Reqdo,Indenizaçao por Dano Moral,NaN,NaN
14,05/03/2018,Processo Materializado,NaN,Indenizaçao por Dano Moral,Setor Unificado de Cartas Precatórias Cíveis,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Edna Kyoko Kano,NaN,FACEBOOK SERVIÇOS ONLINE DO BRASIL LTDA,0009178-80.2018.8.26.0021,Reqdo,Indenizaçao por Dano Moral,NaN,NaN
15,05/03/2018,Distribuído Livremente (por Sorteio) (moviment...,NaN,Indenizaçao por Dano Moral,Setor Unificado de Cartas Precatórias Cíveis,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Edna Kyoko Kano,NaN,FACEBOOK SERVIÇOS ONLINE DO BRASIL LTDA,0009178-80.2018.8.26.0021,Reqdo,Indenizaçao por Dano Moral,NaN,NaN


In [133]:
df_movimenta.to_excel('resultados/processos_teste_tjsp_29_abril_terceira_fase.xlsx',sheet_name='Sheet1')